# Análisis de sentimientos

En esta sección se realiza un análisis de sentimiento de tweets, como un ejemplo del uso de modelos de lenguaje previamente entrenado. En este caso, se utiliza el modelo [Distilbert](https://arxiv.org/abs/1910.01108) para el análisis de sentimientos. Este modelo es una versión más ligera del modelo [BERT](https://arxiv.org/abs/1810.04805), que es un modelo de lenguaje previo a la entrada que se ha utilizado con muy buenos resultados en diferentes tareas de procesamiento del lenguaje natural, como [Análisis de sentimientos](https://arxiv.org/abs/1905.05583), [Clasificación de texto](https://arxiv.org/abs/1904.09077) o [Extracción de información](https://arxiv.org/abs/1906.05237).

En este caso, se utiliza el modelo previamente entrenado para el análisis de sentimientos en inglés.

## Carga del conjunto de datos`

Usaremos la librería [datasets](https://huggingface.co/docs/datasets/) para cargar el `dataset` de los tweets. Esta La librería te permite cargar `datasets` de diferentes fuentes como [Hugging Face Hub](https://huggingface.co/datasets), [Amazon AWS](https://docs.aws.amazon.com/es_es/marketplace/latest/userguide/datasets.html) o [Google Cloud](https://cloud.google.com/ai-platform/training/docs/datasets). En este caso, cargaremos el `dataset` de tuits desde [Hugging Face Hub](https://huggingface.co/datasets/dair-ai/emotion).


In [1]:
# Instalamos las librerias que vamos a usar

import os
os.environ["WANDB_DISABLED"] = "true"

%pip install -U transformers datasets evaluate accelerate scikit-learn accuracy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 12.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [2]:
import datasets

# Cargamos el Dataset
dataset = datasets.load_dataset('dair-ai/emotion')

# Mostramos los datos de ejemplo

dataset['train'][0]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'text': 'i didnt feel humiliated', 'label': 0}

Podemos ver que cada registro del `dataset` contiene el texto del tweet y el sentimiento asociado. En este caso, el sentimiento está codificado con un entero entre 0 y 5, donde 0 corresponde a la tristeza, 1 a Alegria, 2 para amor, 3 a la ira, 4 para miedo y 5 para sorpresa.

## Preparación del `dataset`

En este caso, el `dataset` ya está dividido en conjuntos de entrenamiento, prueba y validación. El siguiente paso es preparar el `dataset` para el entrenamiento del modelo. En este caso, el modelo que utilizaremos es el modelo [BERT](https://arxiv.org/abs/1810.04805). Este modelo requiere que el texto sea tokenizado y los tokens estén codificados con sus identificadores numéricos correspondientes. Para hacer esto, utilizaremos un tokenizador DistilBERT previamente entrenado.

In [3]:
# importamos el tokenizador de DistilBERT
from transformers import AutoTokenizer

# Cargamos el tokenizador
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

# Mostramos un ejemplo de tokenización
tokenizer.tokenize('FC Barcelona is fucked this year')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

['fc', 'barcelona', 'is', 'fucked', 'this', 'year']

In [4]:
# Definimos una función para preprocesar el texto.
# Truncamos los textos para asegurarnos de que no excedan el máximo tamaño de entrada deDistilBert

def tokenize(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

Para aplicar la tokenización, usaremos la función `map` de `datasets`. Esta función permite aplicar una función a cada registro del `dataset`. En este caso, la función que aplicaremos es la función `tokenize` que hemos definido anteriormente. Nosotros también usaremos `batched=True` para indicar que la función se aplicará a todo `dataset` en bloques.

In [5]:
dades_tokenitzades = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

## Evaluación

Para evaluar el modelo, debemos cargar el método que utilizaremos para la evaluación. En este caso usaremos la métrica `accuracy` del módulo `evaluate` de HuggingFace.

También definiremos una función para calcular las métricas del modelo. Esta función se utilizará para evaluar el modelo después de cada época.

In [6]:
import evaluate

accuracy = evaluate.load('accuracy')

In [7]:
# Definimos una función para calcular la precisión del modelo

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Definición de etiquetas

Antes de entrenar el modelo, debemos crear un diccionario que traduzca los identificadores numéricos del sentimiento a sus etiquetas correspondientes y viceversa.

In [8]:
id_a_etiqueta = {
    0: "SADNESS",
    1: "JOY",
    2: "LOVE",
    3: "ANGER",
    4: "FEAR",
    5: "SUPRISE"
}

etiqueta_a_id = {
    "SADNESS": 0,
    "JOY": 1,
    "LOVE": 2,
    "ANGER": 3,
    "FEAR": 4,
    "SUPRISE": 5
}

## Fine tuning del modelo

El proceso de ajuste fino del modelo es entrenar el modelo con nuestro `dataset`. Esto permite que el modelo se adapte mejor a nuestros datos y mejore su rendimiento.

Necesitamos definir la función de optimización, el tamaño de los bloques y el número de épocas.

In [9]:
BATCH_SIZE = 16
NUM_EPOCHS = 1

Ahora podemos cargar el modelo previamente entrenado y hacer el fine tuning. Usaremos `AutoModelForSequenceClassification` y agregaremos las etiquetas que hemos definido previamente.

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert/distilbert-base-uncased',
    num_labels=len(etiqueta_a_id),
    id2label=id_a_etiqueta,
    label2id=etiqueta_a_id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Para evaluar el modelo debemos definir un objeto `TrainingArguments` con los parámetros de entrenamiento. Podemos incluir el número de épocas, el tamaño de los bloques, el tamaño del lote, la tasa de aprendizaje, etc.

In [11]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer",
    eval_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Ahora podemos entrenar el modelo, utilizando el `trainer`.

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dades_tokenitzades['train'],
    eval_dataset=dades_tokenitzades['validation'],
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.205400,0.179499,0.927000


TrainOutput(global_step=1000, training_loss=0.379160888671875, metrics={'train_runtime': 728.165, 'train_samples_per_second': 21.973, 'train_steps_per_second': 1.373, 'total_flos': 2119629570048000.0, 'train_loss': 0.379160888671875, 'epoch': 1.0})

## Inferencia

Para hacer inferencia usando el modelo, crearemos una pipeline Huggingface. Esta pipeline utilizará el modelo y el tokenizador que importamos anteriormente.

Luego usaremos la pipeline para hacer una inferencia con un texto de ejemplo.

In [13]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
print(classifier("Suddenly, I'm not half the man I used to be, There's a shadow hanging over me, Oh, yesterday came suddenly"))
print(classifier("Don't stop me now. I'm havin' such a good time, I'm havin' a ball. If you wanna have a good time, just give me a call"))
print(classifier("Remember those who win the game. Lose the love they sought to gain. In debentures of quality. And dubious integrity. Their small town eyes will gape at you. In dull surprise when payment due. Exceeds accounts received. At seventeen"))
print(classifier("You got your bitches with the silicone injections. Crystal meth and yeast infections. Bleached blond hair, collagen lip injections. Who are you to criticize my intentions?. Got your subtle, manipulative devices. Just like you, I got my vices. I got a thought that would be nice. I'd like to crush your head, tight in my vice. Pain"))

Device set to use cuda:0


[{'label': 'ANGER', 'score': 0.34769198298454285}]
[{'label': 'JOY', 'score': 0.9859300255775452}]
[{'label': 'ANGER', 'score': 0.48408079147338867}]
[{'label': 'ANGER', 'score': 0.9862304329872131}]
